## Race condition

In [4]:
import logging
import threading
import time
import concurrent.futures

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        time.sleep(1)   # simulates a delay before the computation
        local_copy = self.value
        local_copy += 1
        time.sleep(1)   # simulates a delay during the computation
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)') # Answer should be 2!

19:08:53: Testing update. Starting value is 0.
19:08:53: Thread 0: starting update
19:08:53: Thread 1: starting update
19:08:55: Thread 0: finishing update
19:08:55: Thread 1: finishing update
19:08:55: Testing update. Ending value is 1.


Finished in 2.03 second(s)


## Locks

In [5]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        time.sleep(1)   # simulates a delay before the computation
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(1)   # simulates a delay during the computation
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)


logging.getLogger().setLevel(logging.DEBUG)
start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.locked_update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')


19:09:23: Testing update. Starting value is 0.
19:09:23: Thread 0: starting update
19:09:23: Thread 1: starting update
19:09:23: Thread 0 about to lock
19:09:23: Thread 1 about to lock
19:09:24: Thread 1 has lock
19:09:25: Thread 1 about to release lock
19:09:25: Thread 1 after release
19:09:25: Thread 1: finishing update
19:09:25: Thread 0 has lock
19:09:26: Thread 0 about to release lock
19:09:26: Thread 0 after release
19:09:26: Thread 0: finishing update
19:09:26: Testing update. Ending value is 2.


Finished in 3.04 second(s)
